### Import Libs :

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, count, when, hour, dayofweek, month
from dotenv import load_dotenv
import os

load_dotenv()

True

### Create the spark Session:

In [ ]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("App")
    .config(
        "spark.jars",
        "/media/rachid/d70e3dc6-74e7-4c87-96bc-e4c3689c979a/lmobrmij/Projects/Smart-LogiTrack-Syst-me-Pr-dictif-de-Transport-Urbain-ETA-/postgresql-42.7.3.jar"
    )
    .getOrCreate()
)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/12 10:29:50 WARN Utils: Your hostname, rachid-Latitude-5580, resolves to a loopback address: 127.0.1.1; using 192.168.68.108 instead (on interface wlp1s0)
26/01/12 10:29:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
26/01/12 10:30:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/12 10:30:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Read Data :

In [17]:
def ReadDataset(file_path):
    df = spark.read.parquet(file_path)
    if df is None:
        raise Exception("Failed to read dataset")
    else :
        return df

file_path = '../data/Bronze/dataset.parquet'
df = ReadDataset(file_path)
df.show(1)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|       1| 2025-01-01 00:18:38|  2025-01-01 00:26:59|              1|          1.6|         1|                 N|         229|    

### Show Data :

In [18]:
df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|       1| 2025-01-01 00:18:38|  2025-01-01 00:26:59|              1|          1.6|         1|                 N|         229|    

### Display Type and names of columns :

In [19]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)
 |-- cbd_congestion_fee: double (nullable = true)



### Get Number of columns :

In [20]:
num_rows = df.count()
print(num_rows)

3475226


### Check for null values :

In [21]:
def CheckNull(Data):
    num_rows = Data.count()       
    Columns_list = Data.columns

    for c in Columns_list:
        num_null = Data.filter(col(c).isNull()).count()
        if num_null > 0:
            null_percent = (num_null / num_rows) * 100
            print(f"Column {c} has {num_null} null values ({null_percent:.2f}%)")
            
            if null_percent < 5:
                Data = Data.na.drop(subset=[c])
            else:
                try:
                    mean_value = Data.select(mean(c)).collect()[0][0]
                    Data = Data.fillna({c: mean_value})
                except:
                    mode_value = Data.groupBy(c).count().orderBy(col("count").desc()).first()[0]
                    Data = Data.fillna({c: mode_value})
    return df

df = CheckNull(df)
            

Column passenger_count has 540149 null values (15.54%)
Column RatecodeID has 540149 null values (15.54%)
Column store_and_fwd_flag has 540149 null values (15.54%)
Column congestion_surcharge has 540149 null values (15.54%)
Column Airport_fee has 540149 null values (15.54%)


In [22]:
CheckNull(df)

Column passenger_count has 540149 null values (15.54%)
Column RatecodeID has 540149 null values (15.54%)
Column store_and_fwd_flag has 540149 null values (15.54%)
Column congestion_surcharge has 540149 null values (15.54%)
Column Airport_fee has 540149 null values (15.54%)


DataFrame[VendorID: int, tpep_pickup_datetime: timestamp_ntz, tpep_dropoff_datetime: timestamp_ntz, passenger_count: bigint, trip_distance: double, RatecodeID: bigint, store_and_fwd_flag: string, PULocationID: int, DOLocationID: int, payment_type: bigint, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, improvement_surcharge: double, total_amount: double, congestion_surcharge: double, Airport_fee: double, cbd_congestion_fee: double]

### Check Duplicate values :

In [23]:
def CheckDuplicated(Data):
    num_rows = Data.count()
    num_rows_no_duplicate = Data.distinct().count()
    num_duplicate_values = num_rows - num_rows_no_duplicate
    if num_duplicate_values == 0:
        print("you don't have any duplicated values !!")
    else:
        Data = Data.distinct()
        return Data
    return Data

df = CheckDuplicated(df)
df.show()

you don't have any duplicated values !!
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|       1| 2025-01-01 00:18:38|  2025-01-01 00:26:59|              1|          1.6|       

### Time Converting:

In [24]:
def Time_Cleaning(Data):
    Data = Data.withColumn("Durée", col("tpep_dropoff_datetime") - col("tpep_pickup_datetime"))
    Data = Data.withColumn("Durée_minutes", (col("Durée").cast("long")/60).cast("int"))

    Data = Data.withColumn("pickup_hour", hour(col("tpep_pickup_datetime")))
    Data = Data.withColumn("pickup_day_week", dayofweek(col("tpep_pickup_datetime")))
    Data = Data.withColumn("pickup_month", month(col("tpep_pickup_datetime")))

    Data = Data.drop("Durée", "tpep_dropoff_datetime", "tpep_pickup_datetime")
    return Data

df = Time_Cleaning(df)
df.show()

+--------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+-------------+-----------+---------------+------------+
|VendorID|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|Durée_minutes|pickup_hour|pickup_day_week|pickup_month|
+--------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+-------------+-----------+---------------+------------+
|       1|              1|          1.6|         1|                 N|         229|         23

### Outlier trips filtering:

In [32]:
# Distance ≤ 0 ou > 200 miles | Durée ≤ 0 minutes | Passagers ≤ 0
from pyspark.sql.functions import col

def Outliers_filtering(Data):
    Data = Data.filter(
        (col("trip_distance") > 0) & 
        (col("trip_distance") <= 200) & 
        (col("passenger_count") > 0) & 
        (col("Durée_minutes") > 0) &
        (col("fare_amount") > 0) &
        (col("total_amount") > 0) &
        (col("Durée_minutes") <= 240) &
        (col("RatecodeID") != 99)
    )
    return Data

df = Outliers_filtering(df)
df.describe().show()
df.write.mode('overwrite').format("parquet").save("../data/Silver")

df_get = ReadDataset('../data/Silver/')
df_get.describe().show()
df_get.count()

+-------+-------------------+------------------+------------------+-------------------+------------------+-----------------+------------------+------------------+------------------+------------------+-------------------+-----------------+------------------+---------------------+------------------+--------------------+-------------------+-------------------+------------------+------------------+------------------+------------------+
|summary|           VendorID|   passenger_count|     trip_distance|         RatecodeID|store_and_fwd_flag|     PULocationID|      DOLocationID|      payment_type|       fare_amount|             extra|            mta_tax|       tip_amount|      tolls_amount|improvement_surcharge|      total_amount|congestion_surcharge|        Airport_fee| cbd_congestion_fee|     Durée_minutes|       pickup_hour|   pickup_day_week|      pickup_month|
+-------+-------------------+------------------+------------------+-------------------+------------------+-----------------+----

+-------+-------------------+------------------+------------------+-------------------+------------------+-----------------+------------------+------------------+------------------+------------------+-------------------+-----------------+------------------+---------------------+------------------+--------------------+-------------------+-------------------+------------------+------------------+------------------+------------------+
|summary|           VendorID|   passenger_count|     trip_distance|         RatecodeID|store_and_fwd_flag|     PULocationID|      DOLocationID|      payment_type|       fare_amount|             extra|            mta_tax|       tip_amount|      tolls_amount|improvement_surcharge|      total_amount|congestion_surcharge|        Airport_fee| cbd_congestion_fee|     Durée_minutes|       pickup_hour|   pickup_day_week|      pickup_month|
+-------+-------------------+------------------+------------------+-------------------+------------------+-----------------+----

2766698

### Save New Data Like a Silver Data in local And data base : 

In [36]:
# save it in local
def Silver_Local(Data_i):
    Data_i.write.mode('overwrite').format("parquet").save("/media/rachid/d70e3dc6-74e7-4c87-96bc-e4c3689c979a/lmobrmij/Projects/Smart-LogiTrack-Syst-me-Pr-dictif-de-Transport-Urbain-ETA-/ml/data/Silver")


#### Check if data saved With Succe:

In [37]:
df_get = ReadDataset('../data/Silver/')
df_get.describe().show()
df_get.count()

+-------+-------------------+------------------+------------------+-------------------+------------------+-----------------+------------------+------------------+------------------+------------------+-------------------+-----------------+------------------+---------------------+------------------+--------------------+-------------------+-------------------+------------------+------------------+------------------+------------------+
|summary|           VendorID|   passenger_count|     trip_distance|         RatecodeID|store_and_fwd_flag|     PULocationID|      DOLocationID|      payment_type|       fare_amount|             extra|            mta_tax|       tip_amount|      tolls_amount|improvement_surcharge|      total_amount|congestion_surcharge|        Airport_fee| cbd_congestion_fee|     Durée_minutes|       pickup_hour|   pickup_day_week|      pickup_month|
+-------+-------------------+------------------+------------------+-------------------+------------------+-----------------+----

2766698

#### Save Data in db:

In [ ]:
jdbc_url = f"jdbc:postgresql://{os.getenv('host')}:{os.getenv('port')}/{os.getenv('database')}"

connection_properties = {
    "user": os.getenv('user'),
    "password": os.getenv('password'),
    "driver": "org.postgresql.Driver"
}

df_load = spark.read.parquet("../data/Silver/")
df_load.write.jdbc(url=jdbc_url, table="silver_data", mode="overwrite", properties=connection_properties)

26/01/12 10:30:22 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
